In [ ]:
# 킬링 끝음절 리스트 추가
# 비속어, 한글자 입력 시 다시 입력할 수 있게 하기
# 첫 음절만 같게 하고 띄어쓰기, 특수문자 입력 시 다시 입력
# 끝음절이 자음 또는 모음 하나만 있을 경우 종료

In [ ]:
import requests
import json
import random
import re

# 우리말샘 사전 API
api = 'API 발급 후 입력'

# 킬링 끝음절 리스트 지정
ban_last_chars = ['녘', '읊', '튕', '듈', '잚', '앎', '늪', '흙', '맑', '쳑', '딩', '팅', '늬', '뺌', '섯', '뺌',
                  '켬', '쁨', '찧', '앓', '륨', '뜀', '찝', '짊', '얽', '젖', '픔', '륨', '슘', '듐', '츔' '썹']
# 비속어 리스트 지정
ban_words = ['바보', '멍청이', '병신', '시발', '시바', '빙시', '븅신', '씨발', '씨바', '존나', '느금마', '개새끼', '지랄']
# 사용자가 입력한 단어 리스트트
said_y = []

# 첫 입력
query = input("단어를 입력하세요(종료를 원하면 '그만'을 입력해주세요.): ")
# 두 글자 이상으로 단어 최소 길이 제한
while len(query) <= 1:
    query = input("두 글자 이상의 단어를 입력해주세요(종료를 원하면 '그만'을 입력해주세요.): ")
# 비속어 제한한
if query in ban_words:
    print("비속어는 사용할 수 없습니다. 올바른 단어를 입력해주세요.")

# 사용자가 입력한 단어 리스트에 추가가  
said_y.append(query)


while True:
    # 단어 검색 요청
    url = f"https://opendict.korean.go.kr/api/search?certkey_no=6828&key={api}&target_type=search&req_type=json&part=word&q={query[-1]}&sort=date&start=1&num=100&advanced=y&method=start&type1=word&pos=1"
    response = requests.get(url).text
    response = json.loads(response)

    # 이용할 단어 담을 리스트 생성
    word_list = []

    # word_list 리스트에 단어 추가가
    for i in range(100):
        try:
            a = response['channel']['item'][i]['word']
            if a not in said_y:
                word_list.append(a)
        except:
            pass
        
        # 킬링 끝음절, 하이픈(-) 포함 단어 필터링링
        filtered_words = [
            w for w in word_list
            if w[-1] not in ban_last_chars
            and '-' not in w
        ]

    # com이 더이상 이을 수 있는 단어가 없을 때
    if not word_list:
        print('더 이상 이어지는 단어가 없습니다. 당신의 승리!')
        break
    # 필터링 된 단어에서 랜덤으로 추출
    com = random.choice(filtered_words)
    # 사용자 입력 단어와 com의 단어 출력
    print('you:', query, '|', 'com:', com, '\n', '-'*20)
    # com이 사용한 단어 리스트에 추가가
    said_y.append(com)

    # 두 번째 이상의 입력/입력해야할 첫 글자 언급
    query = input(f"{com[-1]}로 시작하는 단어를 입력해주세요(종료를 원하면 '그만'을 입력해주세요.): ")
    
    # 사용자 의지로 게임 종료
    if query == '그만': 
        print('게임 종료!')
        break
    
    # 잘못된 단어 입력
    if query[0] != com[-1]: 
        print(f'{query} 은(는) {com[-1]}로 시작하지 않습니다. com의 승리!')
        break

    # 중복 입력 금지
    if query in said_y: 
        print(f'{query} 은(는) 이미 입력한 단어입니다. 올바른 단어를 입력해주세요.')
        continue
    
    # 단어 최소 길이 제한
    if len(query) <= 1: 
        print('두 글자 이상의 단어를 입력해주세요.')
        continue

    #비속어 제한
    if query in ban_words:  
        print("비속어는 사용할 수 없습니다. 올바른 단어를 입력해주세요.")
        continue

    # 사용자 입력 단어 리스트에 추가
    said_y.append(query)


더 이상 이어지는 단어가 없습니다. 당신의 승리!
